In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt 

In [2]:
# Default value for amount of intervals per octave
intervals_per_octave = 3

# Default value for amount of octaves
num_of_octaves = 4

# Default value for initial gaussian blur
sigma = 1.6

# Default value for contrast threshold
contrast_threshold = 0.04

# Default value for edge threshold
edge_threshold = 10

# Assumed gaussian blur for input image
initial_sigma = 0.5

# Default value for max interpolation steps
max_interp_steps = 5

# Double image size
double_image_size = True

# Default value for number of layers in octave
num_of_layers_in_octave = 3


In [3]:
def create_initial_image(img, double_image_size, sigma):
    if double_image_size:
        sig_diff = np.sqrt(sigma * sigma - initial_sigma * initial_sigma * 4)
        img = cv2.resize(img, (0, 0), fx=2, fy=2, interpolation=cv2.INTER_LINEAR)
        img = cv2.GaussianBlur(img, (0, 0), sig_diff)
    else:
        sig_diff = np.sqrt(sigma * sigma - initial_sigma * initial_sigma)
        img = cv2.GaussianBlur(img, (0, 0), sig_diff)

    # Convert to float32
    img = img.astype(np.float32)

    return img

In [4]:
img = create_initial_image(cv2.imread('images/sunflowers.jpg', cv2.IMREAD_GRAYSCALE), double_image_size, sigma)

In [5]:
def build_gaussian_pyramid(base, nOctaves, nOctaveLayers, sigma):
    sig = [0] * (nOctaveLayers + 3)
    pyr = [None] * (nOctaves * (nOctaveLayers + 3))  # flat list

    # Precompute Gaussian sigmas
    sig[0] = sigma
    k = 2.0 ** (1.0 / nOctaveLayers)
    for i in range(1, nOctaveLayers + 3):
        sig_prev = (k ** (i - 1)) * sigma
        sig_total = sig_prev * k
        sig[i] = np.sqrt(sig_total ** 2 - sig_prev ** 2)

    for o in range(nOctaves):
        for i in range(nOctaveLayers + 3):
            idx = o * (nOctaveLayers + 3) + i
            if o == 0 and i == 0:
                pyr[idx] = base.copy()
            elif i == 0:
                # Downsample last layer of previous octave
                src = pyr[(o - 1) * (nOctaveLayers + 3) + nOctaveLayers]
                pyr[idx] = cv2.resize(src, (src.shape[1] // 2, src.shape[0] // 2), interpolation=cv2.INTER_NEAREST)
            else:
                src = pyr[o * (nOctaveLayers + 3) + i - 1]
                blurred = cv2.GaussianBlur(src, (0, 0), sig[i], sig[i])
                pyr[idx] = blurred

    return pyr

pyr = build_gaussian_pyramid(img, num_of_octaves, num_of_layers_in_octave, sigma)
print(len(pyr))


24


In [ ]:
# Plot the gaussian pyramid
# Array is len(24). 4 octaves with 6 images each
# Each row is an octave

fig, axes = plt.subplots(4, 6, figsize=(15, 10))
axes = axes.ravel()

for i, ax in enumerate(axes):
    ax.imshow(pyr[i], cmap='gray')
    # Get image dimensions
    height, width = pyr[i].shape
    ax.set_title(f'{width}x{height}')
    ax.axis('off')

plt.tight_layout()
plt.show()